## Welcome to your notebook.


#### Run this cell to connect to your GIS and get started:

In [1]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection
import os
import io
import pandas as pd
gis = GIS("home")

You are logged on as ben_bislens with an administrator role, proceed with caution.


#### Now you are ready to start!

Table creation based on the following sensor alert fields i.e. json body example

{
location: "Site1",  
node_id: "<node-id>",  
block: "<block-id>",  
level: 2,  
ward: "C2E",  
asset_type: "plank",  
asset_id: "C2E-208",  
alarm_code: 3,  
object_name: "early_deflection_alert",   
description: "Early deflection alert",  
present_value: 6.0,  
threshold_value: 6.0,  
min_value: -250,  
max_value: 2,  
resolution: 0.1,  
units: "milimetre",  
alarm_status: "InAlarm",  
event_state: "HighLimit",  
alarm_date: YYYY-MM-DDT00:00:00.000",  
device_type: "ultrasonic distance sensor"  
}

def create_hosted_table_from_scratch():
    """Create a hosted table from scratch using add_to_definition"""
    
    # Parameters for creating empty feature service
    service_name = "SensorDataService" # This value is hardcoded in function_app.py (line 156) change it there too!
    
    # Create empty feature service
    empty_service = gis.content.create_service(
        name=service_name,
        has_static_data=False,
        max_record_count=1000,
        capabilities="Query,Create,Update,Delete,Editing",
        service_description="Sensor data collection service",
        tags=["sensor", "building", "monitoring"],
        snippet="Building sensor data"
    )
    
    # Define table structure based on your sensor JSON
    table_definition = {
        "type": "Table",
        "name": "SensorReadings",
        "description": "Building sensor readings",
        "hasAttachments": False,
        "fields": [
            {
                "name": "OBJECTID",
                "type": "esriFieldTypeOID",
                "alias": "OBJECTID",
                "nullable": False,
                "editable": False
            },
            {
                "name": "location",
                "type": "esriFieldTypeString",
                "alias": "Location",
                "length": 100,
                "nullable": True,
                "editable": True
            },
            {
                "name": "node_id",
                "type": "esriFieldTypeString",
                "alias": "Node ID",
                "length": 50,
                "nullable": True,
                "editable": True
            },
            {
                "name": "block_id",
                "type": "esriFieldTypeString",
                "alias": "Block ID",
                "length": 50,
                "nullable": True,
                "editable": True
            },
            {
                "name": "level",
                "type": "esriFieldTypeInteger",
                "alias": "Level",
                "nullable": True,
                "editable": True
            },
            {
                "name": "ward",
                "type": "esriFieldTypeString",
                "alias": "Ward",
                "length": 10,
                "nullable": True,
                "editable": True
            },
            {
                "name": "asset_type",
                "type": "esriFieldTypeString",
                "alias": "Asset Type",
                "length": 50,
                "nullable": True,
                "editable": True
            },
            {
                "name": "asset_id",
                "type": "esriFieldTypeString",
                "alias": "Asset ID",
                "length": 50,
                "nullable": True,
                "editable": True
            },
            {
                "name": "alarm_code",
                "type": "esriFieldTypeInteger",
                "alias": "Alarm Code",
                "nullable": True,
                "editable": True
            },
            {
                "name": "object_name",
                "type": "esriFieldTypeString",
                "alias": "Object Name",
                "length": 100,
                "nullable": True,
                "editable": True
            },
            {
                "name": "description",
                "type": "esriFieldTypeString",
                "alias": "Description",
                "length": 255,
                "nullable": True,
                "editable": True
            },
            {
                "name": "present_value",
                "type": "esriFieldTypeDouble",
                "alias": "Present Value",
                "nullable": True,
                "editable": True
            },
            {
                "name": "threshold_value",
                "type": "esriFieldTypeDouble",
                "alias": "Threshold Value",
                "nullable": True,
                "editable": True
            },
            {
                "name": "min_value",
                "type": "esriFieldTypeDouble",
                "alias": "Min Value",
                "nullable": True,
                "editable": True
            },
            {
                "name": "max_value",
                "type": "esriFieldTypeDouble",
                "alias": "Max Value",
                "nullable": True,
                "editable": True
            },
            {
                "name": "resolution",
                "type": "esriFieldTypeDouble",
                "alias": "Resolution",
                "nullable": True,
                "editable": True
            },
            {
                "name": "units",
                "type": "esriFieldTypeString",
                "alias": "Units",
                "length": 20,
                "nullable": True,
                "editable": True
            },
            {
                "name": "alarm_status",
                "type": "esriFieldTypeString",
                "alias": "Alarm Status",
                "length": 20,
                "nullable": True,
                "editable": True
            },
            {
                "name": "event_state",
                "type": "esriFieldTypeString",
                "alias": "Event State",
                "length": 20,
                "nullable": True,
                "editable": True
            },
            {
                "name": "alarm_date",
                "type": "esriFieldTypeDate",
                "alias": "Alarm Date",
                "nullable": True,
                "editable": True
            },
            {
                "name": "device_type",
                "type": "esriFieldTypeString",
                "alias": "Device Type",
                "length": 100,
                "nullable": True,
                "editable": True
            }
        ]
    }
    
    # Get FeatureLayerCollection and add table definition
    flc = FeatureLayerCollection.fromitem(empty_service)
    result = flc.manager.add_to_definition({"tables": [table_definition]})
    
    print(f"Table creation result: {result}")
    print(f"Service URL: {empty_service.url}")
    print(f"Service ID: {empty_service.id}")
    
    return empty_service


Why Your Original Approach Doesn't Work
The fundamental issue is that tables created using add_to_definition don't have ArcGIS EnterpriseGeodata2use the same field behavior as tables created through the standard ArcGIS Online workflows. The AGOL editing interface expects fields to be configured in a specific way that add_to_definition doesn't replicate properly.
Recommended Solution
Use the CSV template approach (Approach 1 in the code above). This method:

Creates the table structure exactly as ArcGIS Online expects
Uses the same field creation workflow as the AGOL UI
Ensures proper default value behavior
Handles nullable fields correctly

In [6]:
# APPROACH 1: Create from CSV Template
# This mimics how AGOL normally creates tables and is most reliable

def create_table_from_csv_template():
    """Create table by uploading a CSV template with proper structure"""
    
    import tempfile
    import os
    import pandas as pd
    
    # Create a sample CSV with your desired structure and one sample row
    sample_data = {
        'location': ['BIS Towers'],
        'node_id': ['NODE001'],
        'block_id': ['BIS001'],
        'level_code': [1],
        'ward': ['A'],
        'asset_type': ['Temperature Sensor'],
        'asset_id': ['TEMP001'],
        'alarm_code': [1],
        'object_name': ['Sample Object'],
        'description': ['Sample description'],
        'present_value': [23.5],
        'threshold_value': [25.0],
        'min_value': [10.0],
        'max_value': [40.0],
        'resolution': [0.1],
        'units': ['Celsius'],
        'alarm_status': ['Normal'],
        'event_state': ['Active'],
        'alarm_date': ['2025-07-24 09:30:00'],  # YYYY-MM-DD HH:MM:SS
        'device_type': ['Digital Sensor']
    }
    
    # Create DataFrame
    df = pd.DataFrame(sample_data)
    
    # Create temporary file
    with tempfile.NamedTemporaryFile(mode='w', suffix='.csv', delete=False) as temp_file:
        df.to_csv(temp_file, index=False)
        temp_file_path = temp_file.name
    
    try:
        # Upload CSV as item using file path
        csv_item = gis.content.add({
            'title': 'SensorDataService',
            'type': 'CSV',
            'tags': ['sensor', 'template'],
            'description': 'Template for sensor data table'
        }, data=temp_file_path)
        
        print(f"✅ CSV item created: {csv_item.title}")
        
        # Publish as hosted table
        published_table = csv_item.publish()
        print(f"✅ Table published: {published_table.title}")
        
        # Delete the sample row
        table_layer = published_table.tables[0]
        all_features = table_layer.query()
        if all_features.features:
            # Delete all existing features (our template row)
            object_ids = [f.attributes['ObjectId'] for f in all_features.features]
            delete_result = table_layer.edit_features(deletes=object_ids)
            print(f"✅ Sample row deleted: {delete_result}")
        
        # Clean up the CSV item
        csv_item.delete()
        print("✅ Temporary CSV item cleaned up")
        
        return published_table
        
    finally:
        # Clean up temporary file
        if os.path.exists(temp_file_path):
            os.unlink(temp_file_path)

In [7]:
# Execute the function
service_item = create_table_from_csv_template()

/tmp/ipykernel_22/618710702.py:2: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  service_item = create_table_from_csv_template()
/opt/conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


✅ CSV item created: SensorDataService


/opt/conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'geocode.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'services-eu1.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


✅ Table published: SensorDataService
✅ Sample row deleted: {'addResults': [], 'updateResults': [], 'deleteResults': [{'objectId': 1, 'uniqueId': 1, 'globalId': None, 'success': True}]}
✅ Temporary CSV item cleaned up


## Table Tools!

In [ ]:
#from arcgis.gis import GIS
from arcgis.features import FeatureLayer
import sys

# Connect to ArcGIS Online
#gis = GIS("https://www.arcgis.com", "YOUR_USERNAME", "YOUR_PASSWORD")  # <-- PLACEHOLDER

# Hosted table REST URL (replace placeholders)
hosted_table_url = "https://services-eu1.arcgis.com/veDTgAL7B9EBogdG/arcgis/rest/services/SensorDataService/FeatureServer/0"  # <-- PLACEHOLDERS

try:
    table = FeatureLayer(hosted_table_url, gis)
    table_name = table.properties.name
    count_result = table.query(where="1=1", return_count_only=True)
    print(f"Table name: {table_name}")
    print(f"Record count: {count_result}")
except Exception as e:
    print(f"Error accessing or querying table: {e}", file=sys.stderr)


In [ ]:
# get some table properties if needed!
print(table.properties.capabilities)
print(table.properties)
